In [2]:

import pandas as pd
from shapely.geometry import Point, Polygon
from shapely.geometry.polygon import LinearRing
import networkx as nx
import osmnx as ox
import folium
from concave_hull import concave_hull_indexes
import numpy as np
from geopy.distance import geodesic
from PIL import Image
import io

# Un casino de la liste, on suppose que on aura (lat,long,size) dans un dataframe 
casino_location = (48.85286779713016, 2.3278106240257666)
casino_name="EXEMPLE"
size=300
size_limit=1000
if size >size_limit:
    taille=5000
    G = ox.graph_from_point(casino_location, dist=taille, network_type='drive')
    G = ox.add_edge_speeds(G)
else:
    taille=2500
    G = ox.graph_from_point(casino_location, dist=taille, network_type='walk')
    foot_speed=4
center_node = ox.distance.nearest_nodes(G, casino_location[1], casino_location[0])

#Competiteurs 

competitors_data = pd.read_excel('geocoding.xlsx')

def calculate_distance(loc1, loc2):
    return geodesic(loc1, loc2).kilometers


filtered_competitors = []

for _,row in competitors_data.iterrows():
    competitor_location = (row['lat'], row['long'])
    distance = calculate_distance(casino_location, competitor_location)

    if distance <= taille/(2*1000):
        filtered_competitors.append(row)
        
print(len(filtered_competitors))


travel_times = [5,10,15,20,30]
#on fait un isochrone/map pour chaque travel_time
for travel_time in travel_times:
    list_times=[]
    list_distances=[]
    list_names=[]
    list_lat=[]
    list_long=[]

    if size > size_limit:
        for u, v, k, data in G.edges(data=True, keys=True):
            if data['speed_kph'] <=30:
                meters_per_second = (data['speed_kph'] * 0.7) * 1000 / 3600
            elif data['speed_kph'] <=50:
                meters_per_second = (data['speed_kph'] * 0.8) * 1000 / 3600
            else:
                meters_per_second = data['speed_kph'] * 1000 / 3600
            data['time'] = data['length'] / meters_per_second
    else:
        for u, v, k, data in G.edges(data=True, keys=True):
            meters_per_second = foot_speed * 1000 / 3600
            data['time'] = data['length'] / meters_per_second

    subgraph = nx.ego_graph(G, center_node, radius=travel_time*60, distance='time')
    isochrone = ox.graph_to_gdfs(subgraph, edges=False)
    points = [Point(lon, lat) for lon, lat in zip(isochrone['geometry'].x, isochrone['geometry'].y)]

    m = folium.Map(location=casino_location, zoom_start=13)

    point_array = np.array([[point.x, point.y] for point in points])
    idxes = concave_hull_indexes(
        point_array[:, :2],
        #length_threshold=1,
        concavity=1,
    )

    polygon_coords = point_array[idxes, ::-1].tolist()

    polygon = folium.Polygon(locations=polygon_coords, color='blue', fill=True, fill_color='blue', fill_opacity=0.4)

    polygon.add_to(m)

    for _, row in pd.DataFrame(filtered_competitors).iterrows():
        competitor_location = Point(row['long'], row['lat'])
        nearest_node = ox.distance.nearest_nodes(G, row['long'], row['lat']) 
        if nearest_node in subgraph.nodes:
            folium.Marker(location=(row['lat'], row['long']), 
                        popup=f'{travel_time} min', 
                        icon=folium.Icon(color='red')).add_to(m)
            shortest_path = nx.shortest_path(subgraph, source=center_node, target=nearest_node, weight='length', method='dijkstra')
            sum_times = 0
            sum_distances = 0
            for i in range(1,len(shortest_path)):
                edge_data = G.get_edge_data(shortest_path[i-1],shortest_path[i])[0]
                sum_times+=edge_data['time']
                sum_distances+=edge_data['length']
            list_times.append(sum_times)
            list_distances.append(sum_distances)
            list_names.append(row['concurent'])
            list_lat.append(row['lat'])
            list_long.append(row['long'])
        else:
            # Add a blue marker for competitors outside the isochrone
            folium.Marker(location=(row['lat'], row['long']), 
                        icon=folium.Icon(color='blue')).add_to(m)

    df_competitors=pd.DataFrame({'concurent':list_names,'temps trajet' :list_times,'distance':list_distances,'lat':list_lat,'long':list_long})
    df_competitors.to_excel(f"Casino {casino_name}, temps de trajet  {travel_time}.xlsx",index=False)
    # Adding markers for the center and points within the concave hull
    folium.Marker(location=casino_location, popup='Center Point', icon=folium.Icon(color='red')).add_to(m)
    img_data = m._to_png(5)

    img = Image.open(io.BytesIO(img_data))

    img.save(f'map {travel_time}.png')
    # Display the map
    m

52


There was an error managing firefox (error sending request for url (https://github.com/mozilla/geckodriver/releases/latest): error trying to connect: An existing connection was forcibly closed by the remote host. (os error 10054)); using driver found in the cache
There was an error managing firefox (error sending request for url (https://github.com/mozilla/geckodriver/releases/latest): error trying to connect: An existing connection was forcibly closed by the remote host. (os error 10054)); using driver found in the cache
There was an error managing firefox (error sending request for url (https://github.com/mozilla/geckodriver/releases/latest): error trying to connect: An existing connection was forcibly closed by the remote host. (os error 10054)); using driver found in the cache
There was an error managing firefox (error sending request for url (https://github.com/mozilla/geckodriver/releases/latest): error trying to connect: An existing connection was forcibly closed by the remote h

In [3]:
m